# Multi Agent Discovery, Knowledge generation, and Collaboration for Education

This notebook shows how AI Agents for Education that are used by students and teachers can:
1. Store aggregated structured knowledge about students
2. Discover other agents by generating a `thought`
3. Collaborate by using the knowledge of other agents to `personalize` the output

In [1]:
import json
from dotenv import load_dotenv

import chromadb
import openai
import instructor

from agents import StudentAgent, StudentAgentStore, TeacherAgent
from models import EssayAgentKnowledge, MathAgentKnowledge

In [2]:
load_dotenv()

chroma_client = chromadb.Client()
openai_client = openai.OpenAI()
instructor_client = instructor.from_openai(openai_client)

In [3]:
EssayFeedbackAgent = StudentAgent(
    id="1",
    name="EssayFeedbackAgent",
    system_prompt="You are an expert at providing feedback on essays for 8th graders.",
    knowledge_class=EssayAgentKnowledge,
	openai_client=openai_client,
	instructor_client=instructor_client
)

In [4]:
with open(f"../aggregated-feedback-v0/user1-messages.json", "r") as f:
	user1_chat_history = json.load(f)

In [5]:
print(user1_chat_history[0]['content'])

I want feedback on my essay

Grade: 8
Topic: The Importance of Environmental Conservation
Feedback I want: i want to know how i did with structure
Essay: Environmental conservation is very important because it ensures our planet stays healthy. Firstly, protecting natural habitats helps preserve biodiversity. Animals and plants are saved when we look after forests, oceans, and rivers. This is important because it keeps the ecosystem balanced.

Secondly, by conserving the environment, we can fight against global warming. Planting trees and reducing pollution can help control the temperature of the Earth. This is crucial for preventing extreme weather like hurricanes and heatwaves.

However, some might argue that focusing on economic development is more crucial than conservation. They say that building industries and creating jobs is more important. But, this can be short-sighted because if we harm our environment, it will eventually harm us too.

In conclusion, environmental conservation

In [6]:
response = EssayFeedbackAgent.respond(input=user1_chat_history[0]['content'], chat_history=[])
print(response.content)

Your essay on the importance of environmental conservation is well-structured. You have a clear introduction, body paragraphs that expand on the importance of conservation, and a strong conclusion that summarizes your main points. Each paragraph focuses on a specific aspect of environmental conservation, which helps to keep the essay organized and easy to follow.

Additionally, you have used transitional phrases such as "firstly", "secondly", and "however" to guide the reader through your argument. This helps to create a logical flow and connect your ideas together.

One suggestion for improvement would be to provide more specific examples or statistics to support your points. For example, you could include information on the impact of deforestation on biodiversity or the relationship between carbon emissions and global warming. This would strengthen your argument and provide a more persuasive case for the importance of environmental conservation.

Overall, your essay is well-structure

In [7]:
print(EssayFeedbackAgent.knowledge)

introduction=CriteraFeedback(strengths=['clear introduction'], weaknesses=[], suggestions=[]) structure=CriteraFeedback(strengths=['well-structured', 'clear body paragraphs each focusing on specific aspects of conservation', 'use of transitional phrases enhances logical flow and connection of ideas'], weaknesses=[], suggestions=['provide more specific examples or statistics to support points']) argumentation=CriteraFeedback(strengths=['effective communication of the importance of environmental conservation'], weaknesses=[], suggestions=[]) evidence=CriteraFeedback(strengths=[], weaknesses=[], suggestions=['include specific examples or statistics, e.g., impact of deforestation on biodiversity, relationship between carbon emissions and global warming']) conclusion=CriteraFeedback(strengths=['strong conclusion that summarizes main points'], weaknesses=[], suggestions=[])


In [8]:
MathTutorAgent = StudentAgent(
    id="2",
    name="MathTutorAgent",
    system_prompt="You are an expert math tutor for 8th graders.",
    knowledge_class=MathAgentKnowledge,
    openai_client=openai_client,
	instructor_client=instructor_client
)

In [9]:
student_agent_store = StudentAgentStore(chroma_client=chroma_client, openai_client=openai_client)

In [10]:
student_agent_store.add_agents([EssayFeedbackAgent, MathTutorAgent])

In [11]:
LessonPlanGenerator = TeacherAgent(
    id="1",
    name="LessonPlanGenerator",
    system_prompt="You are an expert at generating lesson plans for 8th grade",
    student_agent_store=student_agent_store,
    openai_client=openai_client,
	instructor_client=instructor_client
)

In [12]:
res = LessonPlanGenerator.respond(input="How can I teach students to develop strong thesis statements in their essays? Create a detailed lesson plan.", chat_history=[])

Knowledge needed: student performance in developing thesis statements

Agent: EssayFeedbackAgent

Agent Knowledge Description: Hello, my name is EssayFeedbackAgent, and I am meticulously designed to assist 8th grade students in improving their essay writing skills. My primary function revolves around providing targeted, constructive feedback on student essays to enhance both their writing style and comprehension of essay structure.

**Purpose:**
As an expert in reviewing essays, my aim is to support young writers in developing their abilities to articulate thoughts clearly and effectively through written text. I focus on critical essays aspects, including introductions, structure, argumentation, evidence usage, and conclusions.

**Capabilities:**
1. Analysis of essay components to pinpoint strengths and weaknesses.
2. Provision of concise and actionable suggestions that are separate from the identified weaknesses.
3. Assistance in understanding core essay-writing elements such as thesi

In [13]:
print(res.content)

To help students develop strong thesis statements in their essays, you can create a detailed lesson plan that focuses on the following steps:

1. **Introduction (5 minutes)**
   - Start the lesson by explaining the definition of a thesis statement. Emphasize that it is the central point or argument that the entire essay will revolve around.
   - Provide examples of strong and weak thesis statements to give students a clear understanding of what to aim for.

2. **Understanding the Assignment (10 minutes)**
   - Teach students how to carefully read and analyze the essay prompt or question. Discuss the importance of identifying key keywords and requirements.
   - Have students practice identifying the main topic and purpose of various essay prompts.

3. **Brainstorming Ideas (15 minutes)**
   - Conduct a brainstorming session where students explore different angles or perspectives related to the essay prompt.
   - Encourage students to ask themselves questions like "What am I trying to pr